In [190]:
import stim
import numpy as np
import pymatching
import sinter

In [191]:
data_qubit_coords={(2,0):1,(2,2):2,(2,4):3,(4,4):4,(0,6):5,(2,6):6,(4,6):7,(4,0):8,(6,0):9, (4,2):10, (6,2):11}
ancilla_qubit_coords={(1,4):12, (3,2):13, (3, 5):14, (5,4):15, (5,1):16, (1,6):17, (2,3):18, (4,3):19, (3,0):20, (6,1):21, (1,4):22, (3,2):23, (3, 5):24, (5,4):25, (5,1):26, 27: (4, 1)}
color_face_Z_stabilizer = {12: (2, 3, 5, 6), 13: (1, 2, 3, 4), 14: (3, 4, 6, 7)}
color_face_X_stabilizer = {22: (2, 3, 5, 6), 23: (1, 2, 3, 4), 24: (3, 4, 6, 7)}
edge_X_stabilizer = {17: (5,6), 18: (2,3), 19: (4,10), 20: (1,8)}
surface_face_X_stabilizer = {24: (3, 4, 6, 7), 26: (8, 9, 10, 11)}
surface_face_Z_stabilizer = {12: (2, 3, 5, 6), 13: (1, 2, 3, 4), 15: (4,7,10,11)}
surface_edge_Z_stabilizer = {21: (9,11)}
bell_state_stabilizer={27: (8,10)}

In [192]:
import stim

# 回路の初期化
def initialize_circuit():
    return stim.Circuit()

# キュービットの座標とインデックスの定義
def define_qubit_coords():
    return {
        1: (2, 0), 2: (2, 2), 3: (2, 4), 4: (4, 4), 5: (0, 6),
        6: (2, 6), 7: (4, 6), 8: (4, 0), 9: (6, 0), 10: (4, 2), 11: (6, 2),
        12: (1, 4), 13: (3, 2), 14: (3, 6), 15: (5, 4), 16: (5, 0),
        17: (1, 6), 18: (2, 3), 19: (4, 3), 20: (3, 0), 21: (6, 1),
        22: (4, 1), 23: (2, 1), 24: (3, 4)
    }

# QUBIT_COORDS命令の追加
def append_qubit_coords(circuit, qubit_coords):
    for qubit, (x, y) in qubit_coords.items():
        circuit.append("QUBIT_COORDS", [qubit], [x, y])

# すべてのデータキュービットをリセット
def reset_data_qubits(circuit, data_qubits):
    for qubit in data_qubits:
        circuit.append("R", [qubit])

# すべてのアンシラキュービットをリセット
def reset_ancilla_qubits(circuit, ancilla_qubits):
    for qubit in ancilla_qubits:
        circuit.append("R", [qubit])

# シンドローム測定の追加
def append_syndrome_measurement(circuit, stabilizer, is_X_stabilizer=False, add_detector=False, interval = False):
    # CNOTゲートの適用の前後にHゲートを追加
    if is_X_stabilizer:
        for ancilla in stabilizer.keys():
            circuit.append("H", [ancilla])
        circuit.append("TICK")

    # CNOTゲートの適用
    max_iterations = max(len(data_qubits) for data_qubits in stabilizer.values())
    for i in range(max_iterations):
        for ancilla, data_qubits in stabilizer.items():
            if i < len(data_qubits):
                data_qubit = data_qubits[i]
                if is_X_stabilizer:
                    circuit.append("CX", [ancilla, data_qubit])
                else:
                    circuit.append("CX", [data_qubit, ancilla])
        circuit.append("TICK")

    # CNOTゲートの後にHゲートを追加
    if is_X_stabilizer:
        for ancilla in stabilizer.keys():
            circuit.append("H", [ancilla])
        circuit.append("TICK")

    # MR命令を追加
    for ancilla in stabilizer.keys():
        circuit.append("MR", [ancilla])
    circuit.append("TICK")  # MR命令と他の操作を分離

    # DETECTORの追加
    if add_detector:
        if not interval:
            for idx, ancilla in enumerate(stabilizer.keys()):
                circuit.append("DETECTOR", [stim.target_rec(-len(stabilizer) + idx), stim.target_rec(-len(stabilizer) + idx)])
        else:
            circuit.append("DETECTOR", [stim.target_rec(-4), stim.target_rec(-16)])
            circuit.append("DETECTOR", [stim.target_rec(-6), stim.target_rec(-18)])
            circuit.append("DETECTOR", [stim.target_rec(-3), stim.target_rec(-1), stim.target_rec(-2), stim.target_rec(-15), stim.target_rec(-13), stim.target_rec(-14)])
            circuit.append("DETECTOR", [stim.target_rec(-5), stim.target_rec(-17)])

# メイン関数
if __name__ == "__main__":
    circuit = initialize_circuit()
    qubit_coords = define_qubit_coords()
    data_qubits = [qubit for qubit in qubit_coords.keys() if qubit <= 11]  # データキュービットのリストを作成
    ancilla_qubits = [qubit for qubit in qubit_coords.keys() if qubit > 11]  # アンシラキュービットのリストを作成

    color_face_Z_stabilizer = {
        12: (2, 3, 5, 6), 13: (1, 2, 3, 4), 14: (3, 4, 6, 7)
    }
    color_face_X_stabilizer = {
        12: (2, 3, 5, 6), 13: (1, 2, 3, 4), 14: (3, 4, 6, 7)
    }
    step_1={12: (2, 3, 5, 6), 13: (1, 2, 3, 4, 8, 10), 14: (3, 4, 6, 7), 16: (8, 9, 10, 11)}
    step_2={12: (2, 3, 5, 6), 13: (1, 2, 3, 4, 8, 10), 14: (3, 4, 6, 7), 15: (4, 7, 10, 11), 22: (8, 10), 21: (9, 11)}
    floquet_Z={14: (3, 4, 6, 7), 16: (8, 9, 10, 11), 17: (5, 6), 18: (2, 3), 20: (1, 8), 19: (4, 10)}
    floquet_X={12: (2, 3, 5, 6), 15: (4, 7, 10, 11), 21: (9, 11), 22: (8, 10), 23: (1, 2), 24: (3, 4)}


    append_qubit_coords(circuit, qubit_coords)
    reset_data_qubits(circuit, data_qubits)
    reset_ancilla_qubits(circuit, ancilla_qubits)
    circuit.append("TICK")

    append_syndrome_measurement(circuit, color_face_X_stabilizer, is_X_stabilizer=True, )
    append_syndrome_measurement(circuit, color_face_Z_stabilizer, add_detector=True)
    

    append_syndrome_measurement(circuit, step_1)
    circuit.append("DETECTOR", [stim.target_rec(-4), stim.target_rec(-7)])
    circuit.append("DETECTOR", [stim.target_rec(-3), stim.target_rec(-6)])
    circuit.append("DETECTOR", [stim.target_rec(-2), stim.target_rec(-5)])
    circuit.append("DETECTOR", [stim.target_rec(-1)])

    append_syndrome_measurement(circuit, step_2, is_X_stabilizer=True)
    circuit.append("DETECTOR", [stim.target_rec(-5), stim.target_rec(-2), stim.target_rec(-15)])
    circuit.append("DETECTOR", [stim.target_rec(-16), stim.target_rec(-6)])
    circuit.append("DETECTOR", [stim.target_rec(-14), stim.target_rec(-4)])

    append_syndrome_measurement(circuit, floquet_Z)
    circuit.append("DETECTOR", [stim.target_rec(-3), stim.target_rec(-4), stim.target_rec(-16)])
    circuit.append("DETECTOR", [stim.target_rec(-6), stim.target_rec(-14)])
    circuit.append("DETECTOR", [stim.target_rec(-3), stim.target_rec(-1), stim.target_rec(-2), stim.target_rec(-15)])
    circuit.append("DETECTOR", [stim.target_rec(-5), stim.target_rec(-13)])

    append_syndrome_measurement(circuit, floquet_X, is_X_stabilizer=True)
    circuit.append("DETECTOR", [stim.target_rec(-6), stim.target_rec(-18)])
    circuit.append("DETECTOR", [stim.target_rec(-5), stim.target_rec(-15)])
    circuit.append("DETECTOR", [stim.target_rec(-4), stim.target_rec(-13)])
    circuit.append("DETECTOR", [stim.target_rec(-1), stim.target_rec(-2), stim.target_rec(-3), stim.target_rec(-17)])

    for i in range(3):
        append_syndrome_measurement(circuit, floquet_Z, add_detector=True, interval=True)
        append_syndrome_measurement(circuit, floquet_X, is_X_stabilizer=True, add_detector=True, interval=True)



    # 回路の表示
    print(circuit)

QUBIT_COORDS(2, 0) 1
QUBIT_COORDS(2, 2) 2
QUBIT_COORDS(2, 4) 3
QUBIT_COORDS(4, 4) 4
QUBIT_COORDS(0, 6) 5
QUBIT_COORDS(2, 6) 6
QUBIT_COORDS(4, 6) 7
QUBIT_COORDS(4, 0) 8
QUBIT_COORDS(6, 0) 9
QUBIT_COORDS(4, 2) 10
QUBIT_COORDS(6, 2) 11
QUBIT_COORDS(1, 4) 12
QUBIT_COORDS(3, 2) 13
QUBIT_COORDS(3, 6) 14
QUBIT_COORDS(5, 4) 15
QUBIT_COORDS(5, 0) 16
QUBIT_COORDS(1, 6) 17
QUBIT_COORDS(2, 3) 18
QUBIT_COORDS(4, 3) 19
QUBIT_COORDS(3, 0) 20
QUBIT_COORDS(6, 1) 21
QUBIT_COORDS(4, 1) 22
QUBIT_COORDS(2, 1) 23
QUBIT_COORDS(3, 4) 24
R 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
TICK
H 12 13 14
TICK
CX 12 2 13 1 14 3
TICK
CX 12 3 13 2 14 4
TICK
CX 12 5 13 3 14 6
TICK
CX 12 6 13 4 14 7
TICK
H 12 13 14
TICK
MR 12 13 14
TICK
CX 2 12 1 13 3 14
TICK
CX 3 12 2 13 4 14
TICK
CX 5 12 3 13 6 14
TICK
CX 6 12 4 13 7 14
TICK
MR 12 13 14
TICK
DETECTOR rec[-3] rec[-3]
DETECTOR rec[-2] rec[-2]
DETECTOR rec[-1] rec[-1]
CX 2 12 1 13 3 14 8 16
TICK
CX 3 12 2 13 4 14 9 16
TICK
CX 5 12 3 13 6 14 10 16
TICK
C

In [193]:
diagram = circuit.diagram("timeline-svg")
with open("circuit_timeline.svg", "w") as f:
    f.write(str(diagram))
# SVGからPDFに変換
import cairosvg
cairosvg.svg2pdf(url="circuit_timeline.svg", write_to="circuit_timeline.pdf")

diagram = circuit.diagram("detslice-svg")
with open("circuit_detslice.svg", "w") as f:
    f.write(str(diagram))
# SVGからPDFに変換
import cairosvg
cairosvg.svg2pdf(url="circuit_detslice.svg", write_to="circuit_detslice.pdf")

diagram = circuit.without_noise().diagram(
    "detslice-with-ops-svg", 
    tick=range(0, 100),
)
with open("circuit_detslice-with-ops.svg", "w") as f:
    f.write(str(diagram))
# SVGからPDFに変換
import cairosvg
cairosvg.svg2pdf(url="circuit_detslice-with-ops.svg", write_to="circuit_detslice-with-ops.pdf")

In [194]:
dem = circuit.detector_error_model()
print(repr(dem))

stim.DetectorErrorModel('''
    detector D0
    detector D1
    detector D2
    detector D3
    detector D4
    detector D5
    detector D6
    detector D7
    detector D8
    detector D9
    detector D10
    detector D11
    detector D12
    detector D13
    detector D14
    detector D15
    detector D16
    detector D17
    detector D18
    detector D19
    detector D20
    detector D21
    detector D22
    detector D23
    detector D24
    detector D25
    detector D26
    detector D27
    detector D28
    detector D29
    detector D30
    detector D31
    detector D32
    detector D33
    detector D34
    detector D35
    detector D36
    detector D37
    detector D38
    detector D39
    detector D40
    detector D41
''')


In [197]:
import stim
import ldpc
import numpy as np
from stimbposd import BPOSD

num_shots = 100
d = 5
p = 0.007

sampler = circuit.compile_detector_sampler()
shots, observables = sampler.sample(num_shots, separate_observables=True)

decoder = BPOSD(circuit.detector_error_model(), max_bp_iters=20)

predicted_observables = decoder.decode_batch(shots)
num_mistakes = np.sum(np.any(predicted_observables != observables, axis=1))

print(f"{num_mistakes}/{num_shots}")

IndexError: index 0 is out of bounds for axis 0 with size 0